## Set **seed**

In [ ]:
from sefixlines.utils import set_all_seeds

set_all_seeds()

## Data

In [ ]:
import torchvision.transforms as T
from sefixlines.datasets import ImageClassificationDataset

### **Initial**

In [ ]:
image_paths = []
labels = []

In [ ]:
classes = []

### **Split**

In [ ]:
from sklearn.model_selection import train_test_split

train_image_paths, valid_image_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42) # stratify=labels

### **Augmentations**

In [ ]:
ImageClassificationDataset.augmentation = T.Compose([
    # Добавьте своих аугментаций (по желанию)
])

### Create **Datasets**

In [ ]:
dataset = ImageClassificationDataset(image_paths, labels)

train_set = ImageClassificationDataset(train_image_paths, train_labels, True)
valid_set = ImageClassificationDataset(valid_image_paths, valid_labels)

### ***Visualization***

In [ ]:
dataset.show(classes=classes)

## **Models**

In [ ]:
from torch import nn, optim
from sefixlines.models import Classifier

### *Score*

In [ ]:
scores = dict()

### **Model**: `EfficientNet_B0`

In [ ]:
model_image_size = (224, 224)
ImageClassificationDataset.change_image_size(model_image_size)

In [ ]:
from torchvision import models

model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, len(classes))

In [ ]:
model_wrapped = Classifier(model, "efficientnet_b0")
model_wrapped.fit(train_set, valid_set, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_predictions(valid_set, classes=classes)

### **Model**: `google/vit-base-patch16-224`

In [ ]:
model_id = "google/vit-base-patch16-224"

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_id)
def transform(image):
    return processor(image)['pixel_values']

ImageClassificationDataset.transform = transform

In [ ]:
from sefixlines.utils import CustomOutput
from transformers import AutoModelForImageClassification

model = CustomOutput(
    AutoModelForImageClassification.from_pretrained(
        model_id,
        num_labels=len(classes),
        ignore_mismatched_sizes=True
    )
)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
model_wrapped = Classifier(model, model_id.split('/')[-1], optimizer=optimizer)
model_wrapped.fit(train_set, valid_set, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_predictions(valid_set, classes=classes)

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

## Submission

In [ ]:
test_dir = r""

In [ ]:
import os

test_image_names = os.listdir(test_dir)
test_image_paths = [f"{test_dir}/{image_name}" for image_name in test_image_names]
test_set = ImageClassificationDataset(test_image_paths)

In [ ]:
prediction_class_id = best_model_wrapped.predict(test_set)
prediction_class_names = [classes[class_id] for class_id in prediction_class_id]